# BERT Q/A Model for document search

In [1]:
import os
import re
import glob
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
from tqdm import tqdm
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', 100)

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

In [15]:
def absoluteFilePaths(directory):
    path = []
    files = []
    for dirpath, dirname, filenames in os.walk(directory):
        for f in filenames:
            if not os.path.basename(dirpath).startswith('.'):
                path.append(dirpath)
                files.append(f)
            
    return path, files

In [16]:
# question = "What is attention mechanism"
# true_answer = "The attention mechanism is a part of a neural architecture that enables to dynamically highlight relevant features of the input data, which, in NLP, is typically a sequence of textual elements. It can be applied directly to the raw input or to its higher level representation."

# question = "What is quantum entanglement"
# true_answer = "Quantum Entanglement allows qubits that are separated by incredible distances to interact with each other instantaneously (not limited to the speed of light)."

question = "What are the applications of Face Swapping"
true_answer = "Face swapping has a number of compelling applications in video compositing, transfiguration in portraits, and especially in  identity  protection  as  it  can  replace  faces  in  photographs by ones from a collection of stock images"

In [17]:
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

In [18]:
DIRECTORY = os.getcwd()
locations, documents = absoluteFilePaths(os.path.join(DIRECTORY, 'Google', 'research'))
paths = [os.path.join(loc, doc) for loc, doc in zip(locations, documents)]

In [19]:
paths

['/home/jupyter/nlp_document_finder/Google/research/Attention in Natural Language Processing.pdf',
 '/home/jupyter/nlp_document_finder/Google/research/nlp.pdf',
 '/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf',
 '/home/jupyter/nlp_document_finder/Google/research/quantum_computing.pdf']

In [20]:
def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [21]:
contents = []
for path in paths:
    if path.endswith('.pdf'):
        contents.append(convert_pdf_to_txt(path))
    else: 
        continue

In [22]:
CHUNK_SIZE = 100

## BERT Large Cased SQuAD

In [23]:
#Model
model = BertForQuestionAnswering.from_pretrained('bert-large-cased-whole-word-masking-finetuned-squad')

#Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-cased-whole-word-masking-finetuned-squad')

In [24]:
paths

['/home/jupyter/nlp_document_finder/Google/research/Attention in Natural Language Processing.pdf',
 '/home/jupyter/nlp_document_finder/Google/research/nlp.pdf',
 '/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf',
 '/home/jupyter/nlp_document_finder/Google/research/quantum_computing.pdf']

In [25]:
contents = [re.sub(r'\n', ' ', content) for content in contents]

In [26]:
answers = []
for path, content in tqdm(zip(paths, contents)):
    for i in range(0, len(content.split(" ")), CHUNK_SIZE):
        paragraph = content.split(" ")[i:i+CHUNK_SIZE]
        encoding = tokenizer.encode_plus(text=question, text_pair=paragraph)
        inputs = encoding['input_ids']  #Token embeddings
        sentence_embedding = encoding['token_type_ids']  #Segment embeddings
        tokens = tokenizer.convert_ids_to_tokens(inputs) #input tokens
        
        start_scores, end_scores = model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))

        start_index = torch.argmax(start_scores)
        end_index = torch.argmax(end_scores)
        answer = ' '.join(tokens[start_index:end_index+1])
        if start_index.numpy() < end_index.numpy():
            answers.append([path, answer, i, i+start_index.numpy(), i+end_index.numpy(), (torch.max(start_scores)+torch.max(end_scores)).detach().numpy()])    

4it [04:57, 74.27s/it]


In [27]:
df_answers = pd.DataFrame(data=answers, columns = ['path', 'answer', 'chunk', 'start_loc', 'end_loc', 'logit'])
df_answers = df_answers.sort_values(by=['logit'], ascending=False).reset_index(drop=True)

In [28]:
df_answers.head()

,path,answer,chunk,start_loc,end_loc,logit
0,/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf,A [UNK] process for face manipulation detection where the [UNK] step aims to [UNK] crop and [UNK...,3100,3166,3188,7.7442517
1,/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf,to create a video of the target person doing or saying things,200,299,310,7.47252
2,/home/jupyter/nlp_document_finder/Google/research/quantum_computing.pdf,to show wave nature and particle nature of light,6700,6802,6810,7.1532917
3,/home/jupyter/nlp_document_finder/Google/research/Attention in Natural Language Processing.pdf,applications to several [UNK] including [UNK] [UNK] legal [UNK] and [UNK] [UNK] Paolo,17500,17547,17559,6.718217
4,/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf,Real data [UNK] [UNK] [UNK],6800,6900,6904,6.2739396


In [29]:
for i in range(5):
    print('\033[1m' + df_answers.loc[i, 'path'] + '\033[0m')
    print(" ".join(contents[paths.index(df_answers.loc[i, 'path'])].split(" ")[df_answers.loc[i, 'chunk']: df_answers.loc[i, 'chunk']+CHUNK_SIZE]))
    print()

/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf
A recurrent convolutional model (RCN) was proposed based on the integration of the convolutional network DenseNet [68] and the gated recurrent unit cells [85] to exploit temporal discrepancies across frames (see Fig. 4). The proposed method is tested on the FaceForensics++ data set, which includes 1,000 videos [86], and shows promising results.  Fig. 4. A two-step process for face manipulation detection where the preprocessing step aims to detect, crop and align faces on a sequence of frames and the second step distinguishes manipulated and authentic face images by combining convolutional neural network (CNN) and recurrent neural network (RNN) [84].

/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf
to deepfake technologies. By reviewing the background of deep- fakes and state-of-the-art deepfake detection methods, this study provides a comprehensive overview of deepfake techniques and facilitates

### Compute F1 scores

In [30]:
for i in range(5):
    df_answers.loc[i, 'f1'] = compute_f1(true_answer, df_answers.loc[i, 'answer'])

In [31]:
df_answers.head()

,path,answer,chunk,start_loc,end_loc,logit,f1
0,/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf,A [UNK] process for face manipulation detection where the [UNK] step aims to [UNK] crop and [UNK...,3100,3166,3188,7.7442517,0.153846
1,/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf,to create a video of the target person doing or saying things,200,299,310,7.47252,0.095238
2,/home/jupyter/nlp_document_finder/Google/research/quantum_computing.pdf,to show wave nature and particle nature of light,6700,6802,6810,7.1532917,0.097561
3,/home/jupyter/nlp_document_finder/Google/research/Attention in Natural Language Processing.pdf,applications to several [UNK] including [UNK] [UNK] legal [UNK] and [UNK] [UNK] Paolo,17500,17547,17559,6.718217,0.088889
4,/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf,Real data [UNK] [UNK] [UNK],6800,6900,6904,6.2739396,0.000000


## BERT Base Cased SQuAD 2

In [32]:
#Model
model = BertForQuestionAnswering.from_pretrained('deepset/bert-base-cased-squad2')

#Tokenizer
tokenizer = BertTokenizer.from_pretrained('deepset/bert-base-cased-squad2')

In [33]:
answers = []
for path, content in tqdm(zip(paths, contents)):
    for i in range(0, len(content.split(" ")), CHUNK_SIZE):
        paragraph = content.split(" ")[i:i+CHUNK_SIZE]
        encoding = tokenizer.encode_plus(text=question, text_pair=paragraph)
        inputs = encoding['input_ids']  #Token embeddings
        sentence_embedding = encoding['token_type_ids']  #Segment embeddings
        tokens = tokenizer.convert_ids_to_tokens(inputs) #input tokens
        
        start_scores, end_scores = model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))

        start_index = torch.argmax(start_scores)
        end_index = torch.argmax(end_scores)
        answer = ' '.join(tokens[start_index:end_index+1])
        if start_index.numpy() < end_index.numpy():
            answers.append([path, answer, i, i+start_index.numpy(), i+end_index.numpy(), (torch.max(start_scores)+torch.max(end_scores)).detach().numpy()])    

4it [01:31, 22.90s/it]


In [34]:
df_answers = pd.DataFrame(data=answers, columns = ['path', 'answer', 'chunk', 'start_loc', 'end_loc', 'logit'])
df_answers = df_answers.sort_values(by=['logit'], ascending=False).reset_index(drop=True)

In [35]:
df_answers.head()

,path,answer,chunk,start_loc,end_loc,logit
0,/home/jupyter/nlp_document_finder/Google/research/quantum_computing.pdf,require design and build specific,9100,9206,9210,13.2074375
1,/home/jupyter/nlp_document_finder/Google/research/quantum_computing.pdf,to show wave nature and particle nature of light,6700,6802,6810,12.380996
2,/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf,intelligence services,5300,5363,5364,8.194283
3,/home/jupyter/nlp_document_finder/Google/research/Attention in Natural Language Processing.pdf,to have attention,10100,10200,10202,7.9642606
4,/home/jupyter/nlp_document_finder/Google/research/Attention in Natural Language Processing.pdf,gram attention models for sentence similarity and [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK...,15400,15427,15464,7.942478


In [36]:
for i in range(5):
    print('\033[1m' + df_answers.loc[i, 'path'] + '\033[0m')
    print(" ".join(contents[paths.index(df_answers.loc[i, 'path'])].split(" ")[df_answers.loc[i, 'chunk']: df_answers.loc[i, 'chunk']+CHUNK_SIZE]))
    print()

/home/jupyter/nlp_document_finder/Google/research/quantum_computing.pdf
  description in fact contradicts some predictions of quantum mechanics.     In the face of these two perfectly convincing and contradictory results, there is only  one way out: ask Nature how it works. Till the end of 1970 there was no experimental  result to answer this question. The contradiction discovered by Bell in EPR paper, is  so  subtle  that  it  appears  only  in  a  very  peculiar  situations  that  had  not  been  investigated. And require design and build specific

/home/jupyter/nlp_document_finder/Google/research/quantum_computing.pdf
H   Fully Silvered   Half Silvered  (Beam splitter)                                                                             Fig 3: Example to show wave nature and particle nature of light

/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf
fake contents quickly. Sometimes deepfakes do not need to be spread to massive audience to cause detrimental

### Compute F1 scores

In [37]:
for i in range(5):
    df_answers.loc[i, 'f1'] = compute_f1(true_answer, df_answers.loc[i, 'answer'])

In [38]:
df_answers.head()

,path,answer,chunk,start_loc,end_loc,logit,f1
0,/home/jupyter/nlp_document_finder/Google/research/quantum_computing.pdf,require design and build specific,9100,9206,9210,13.2074375,0.054054
1,/home/jupyter/nlp_document_finder/Google/research/quantum_computing.pdf,to show wave nature and particle nature of light,6700,6802,6810,12.380996,0.097561
2,/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf,intelligence services,5300,5363,5364,8.194283,0.000000
3,/home/jupyter/nlp_document_finder/Google/research/Attention in Natural Language Processing.pdf,to have attention,10100,10200,10202,7.9642606,0.000000
4,/home/jupyter/nlp_document_finder/Google/research/Attention in Natural Language Processing.pdf,gram attention models for sentence similarity and [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK...,15400,15427,15464,7.942478,0.028571


## BERT Large uncased SQuAD

In [25]:
#Model
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

#Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [26]:
answers = []
for path, content in tqdm(zip(paths, contents)):
    for i in range(0, len(content.split(" ")), CHUNK_SIZE):
        paragraph = content.split(" ")[i:i+CHUNK_SIZE]
        encoding = tokenizer.encode_plus(text=question, text_pair=paragraph)
        inputs = encoding['input_ids']  #Token embeddings
        sentence_embedding = encoding['token_type_ids']  #Segment embeddings
        tokens = tokenizer.convert_ids_to_tokens(inputs) #input tokens
        
        start_scores, end_scores = model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))

        start_index = torch.argmax(start_scores)
        end_index = torch.argmax(end_scores)
        answer = ' '.join(tokens[start_index:end_index+1])
        if start_index.numpy() < end_index.numpy():
            answers.append([path, answer, i, i+start_index.numpy(), i+end_index.numpy(), (torch.max(start_scores)+torch.max(end_scores)).detach().numpy()])    

4it [06:10, 92.57s/it] 


In [27]:
df_answers = pd.DataFrame(data=answers, columns = ['path', 'answer', 'chunk', 'start_loc', 'end_loc', 'logit'])
df_answers = df_answers.sort_values(by=['logit'], ascending=False).reset_index(drop=True)

In [28]:
df_answers.head()

,path,answer,chunk,start_loc,end_loc,logit
0,/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf,fake image detection and face video,2300,2310,2315,11.70063
1,/home/jupyter/nlp_document_finder/Google/research/Attention in Natural Language Processing.pdf,[UNK] [UNK] legal [UNK] and [UNK] [UNK] [UNK],17500,17551,17558,11.60136
2,/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf,to detect fake face images,9500,9529,9533,10.294424
3,/home/jupyter/nlp_document_finder/Google/research/Attention in Natural Language Processing.pdf,abusive speech recognition and sentiment,6050,6070,6074,10.155699
4,/home/jupyter/nlp_document_finder/Google/research/Attention in Natural Language Processing.pdf,suggest that the data [UNK] and [UNK] be interpreted in multiple [UNK] [UNK] can be the case whe...,7600,7625,7659,10.107832


In [29]:
for i in range(5):
    print('\033[1m' + df_answers.loc[i, 'path'] + '\033[0m')
    print(" ".join(contents[paths.index(df_answers.loc[i, 'path'])].split(" ")[df_answers.loc[i, 'chunk']: df_answers.loc[i, 'chunk']+CHUNK_SIZE]))
    print()

/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf
fake image detection and face video detection.  requires a large database of real and fake videos to train clas- siﬁcation models. The number of fake videos is increasingly available, but it is still limited in terms of setting a benchmark for validating various detection methods. To address this issue,

/home/jupyter/nlp_document_finder/Google/research/Attention in Natural Language Processing.pdf
He is cur- rently an Associate Professor with the Department of Sciences and Methods for Engineering, Univer- sity of Modena and Reggio Emilia, Modena, Italy. His work focuses on machine learning and artiﬁ- cial intelligence, with applications to several areas, including argumentation mining, legal informatics, and medicine.  Paolo Torroni

/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf
(MIPR) (pp. 384-389). IEEE.  [82] Hsu, C. C., Lee, C. Y., and Zhuang, Y. X. (2018, December). Learning to detect fak

### Compute F1 scores

In [30]:
for i in range(5):
    df_answers.loc[i, 'f1'] = compute_f1(true_answer, df_answers.loc[i, 'answer'])

In [31]:
df_answers.head()

,path,answer,chunk,start_loc,end_loc,logit,f1
0,/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf,fake image detection and face video,2300,2310,2315,11.70063,0.157895
1,/home/jupyter/nlp_document_finder/Google/research/Attention in Natural Language Processing.pdf,[UNK] [UNK] legal [UNK] and [UNK] [UNK] [UNK],17500,17551,17558,11.60136,0.050000
2,/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf,to detect fake face images,9500,9529,9533,10.294424,0.108108
3,/home/jupyter/nlp_document_finder/Google/research/Attention in Natural Language Processing.pdf,abusive speech recognition and sentiment,6050,6070,6074,10.155699,0.054054
4,/home/jupyter/nlp_document_finder/Google/research/Attention in Natural Language Processing.pdf,suggest that the data [UNK] and [UNK] be interpreted in multiple [UNK] [UNK] can be the case whe...,7600,7625,7659,10.107832,0.125000
